# RNN/LSTM 中文影评情感分类实验

这个Notebook实现了使用RNN和LSTM进行中文影评情感分类的任务。实验使用ChnSentiCorp酒店评论数据集，对比了两种模型的性能差异。

## RNN 自回归任务

这部分实现了一个简单的字符级RNN模型用于自回归任务。

In [ ]:
import torch
from torch import nn
import numpy as np
vocab_size=32 # 示例值，实际会根据文本计算
# 定义句子
text = ['hey how are you','good i am fine','have a nice day']

# 连接所有句子并提取唯一字符
chars = set(''.join(text))

# 创建映射整数到字符的字典
int2char = dict(enumerate(chars))

# 创建映射字符到整数的字典
char2int = {char: ind for ind, char in int2char.items()}

print(char2int)
print(int2char)

In [ ]:
# 找出最长句子的长度
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

# 填充
# 循环遍历句子列表，添加空格直到句子长度与最长句子相匹配
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

# 创建列表来存储输入和目标序列
input_seq_str = [] # 使用新变量名以避免与后续的input_seq冲突
target_seq_str = []

for i in range(len(text)):
    # 移除最后一个字符得到输入序列
    input_seq_str.append(text[i][:-1])
    
    # 移除第一个字符得到目标序列
    target_seq_str.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq_str[i], target_seq_str[i]))

In [ ]:
# 将字符序列转换为整数序列
input_seq_int = [] # 使用新变量名
target_seq_int = [] # 使用新变量名
for i in range(len(text)):
    input_seq_int.append([char2int[character] for character in input_seq_str[i]])
    target_seq_int.append([char2int[character] for character in target_seq_str[i]])

# 定义关键变量
dict_size = len(char2int)
seq_len_autoregressive = maxlen - 1 # 使用新变量名
batch_size_autoregressive = len(text) # 使用新变量名

In [ ]:
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # 创建指定输出形状的多维零数组
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # 在相关字符索引处将0替换为1以表示该字符
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

# 对输入序列进行one-hot编码
input_seq_one_hot = one_hot_encode(input_seq_int, dict_size, seq_len_autoregressive, batch_size_autoregressive)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq_one_hot.shape))

In [ ]:
# 将数据从numpy数组转换为PyTorch张量
input_tensor_autoregressive = torch.from_numpy(input_seq_one_hot)
target_tensor_autoregressive = torch.Tensor(target_seq_int)

# 检查是否可以使用GPU
is_cuda = torch.cuda.is_available()

# 如果有GPU，将设备设置为GPU；否则使用CPU
if is_cuda:
    device_autoregressive = torch.device("cuda") # 使用新变量名
    print("GPU is available for autoregressive task")
else:
    device_autoregressive = torch.device("cpu")
    print("GPU not available, CPU used for autoregressive task")

In [ ]:
# 定义模型类
class AutoregressiveModel(nn.Module): # 重命名以避免冲突
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(AutoregressiveModel, self).__init__()

        # 定义一些参数
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = device_autoregressive # 确保模型知道设备

        # 定义层
        # RNN层
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # 全连接层
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(self.device)
        return hidden

In [ ]:
# 用超参数实例化模型
model_autoregressive = AutoregressiveModel(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model_autoregressive = model_autoregressive.to(device_autoregressive)

# 定义超参数
n_epochs_autoregressive = 100
lr_autoregressive =0.01

# 定义损失函数和优化器
criterion_autoregressive = nn.CrossEntropyLoss()
optimizer_autoregressive = torch.optim.Adam(model_autoregressive.parameters(), lr=lr_autoregressive)

In [ ]:
# 训练模型
input_tensor_autoregressive = input_tensor_autoregressive.to(device_autoregressive)
target_tensor_autoregressive = target_tensor_autoregressive.to(device_autoregressive)
for epoch in range(1, n_epochs_autoregressive + 1):
    optimizer_autoregressive.zero_grad() 
    output, hidden = model_autoregressive(input_tensor_autoregressive)
    loss = criterion_autoregressive(output, target_tensor_autoregressive.view(-1).long())
    loss.backward() 
    optimizer_autoregressive.step() 
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs_autoregressive), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

In [ ]:
# 定义预测辅助函数
@torch.no_grad()
def predict_autoregressive(model, character_input_str):
    # 对我们的输入进行one-hot编码以适应模型
    character_int_seq = np.array([[char2int[c] for c in character_input_str]])
    character_one_hot = one_hot_encode(character_int_seq, dict_size, character_int_seq.shape[1], 1)
    character_tensor = torch.from_numpy(character_one_hot)
    character_tensor = character_tensor.to(device_autoregressive)
    
    out, hidden = model(character_tensor)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # 从输出中取概率得分最高的类别
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

def sample_autoregressive(model, out_len, start='hey'):
    model.eval() # 评估模式
    start = start.lower()
    # 首先，遍历起始字符
    chars_list = [ch for ch in start]
    size = out_len - len(chars_list)
    # 现在传入前面的字符并获取新字符
    for _ in range(size):
        # 传递到目前为止生成的整个序列或仅传递最后一个字符/固定窗口
        # 为了简单起见，这里我们传递整个到目前为止的序列
        # 注意：原始代码的predict函数期望一个字符列表，但one_hot_encode期望一个整数列表。
        # predict_autoregressive的输入应该是字符串
        current_sequence_str = "".join(chars_list)
        char, h = predict_autoregressive(model, current_sequence_str)
        chars_list.append(char)

    return ''.join(chars_list)

In [ ]:
# 测试模型
print(sample_autoregressive(model_autoregressive, 15, 'good'))

## 中文影评情感分类任务

这部分实现了使用RNN和LSTM进行中文影评情感分类的实验。

In [ ]:
# 设置随机种子以保证实验的可重复性
import random
import numpy as np
import torch

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("随机种子已设置为", SEED)

# 导入所需库
import gensim
import jieba
import os
from torch.utils.data import Dataset, DataLoader
from torch.functional import F
import pandas as pd 

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 主任务设备
print(f"Using device for sentiment classification: {device}")

# Matplotlib 中文显示设置
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

### 数据预处理与加载

In [ ]:
class RNNDataset(Dataset):
    def __init__(self, data, word2vec_model, seq_len, word_len=100):
        label = data['label'].to_numpy().astype(np.float32)
        self.word2vec_model = word2vec_model
        self.data = [list(jieba.cut(str(sentence))) for sentence in data['review'].to_list()]
        self.label = torch.from_numpy(label).unsqueeze(-1)
        self.seq_len = seq_len
        self.word_len = word_len        

    def preprocess(self, sentence):
        valid_words = [word for word in sentence if word in self.word2vec_model.wv]
        if not valid_words:
            return torch.zeros((self.seq_len, self.word_len), dtype=torch.float32)

        sentence_vectors = np.array([self.word2vec_model.wv[word] for word in valid_words], dtype=np.float32)
        sentence_tensor = torch.from_numpy(sentence_vectors)
        
        current_len = sentence_tensor.shape[0]
        if current_len == 0:
             return torch.zeros((self.seq_len, self.word_len), dtype=torch.float32)

        if current_len > self.seq_len:
            sentence_tensor = sentence_tensor[:self.seq_len, :]
        elif current_len < self.seq_len:
            padding_size = self.seq_len - current_len
            sentence_tensor = F.pad(sentence_tensor, (0, 0, 0, padding_size), 'constant', 0)
        return sentence_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]
        label = self.label[idx]
        return self.preprocess(sentence), label

In [ ]:
def build_word2vec_model(data, word_len, model_dir='./model', model_name='word2vec.model'):
    sentences = [list(jieba.cut(str(sentence))) for sentence in data['review'].to_list()]
    
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_path = os.path.join(model_dir, model_name)
        
    if os.path.exists(model_path):
        print(f"Loading existing Word2Vec model from {model_path}")
        word2vec_model = gensim.models.Word2Vec.load(model_path)
    else:
        print(f"Building new Word2Vec model and saving to {model_path}")
        word2vec_model = gensim.models.Word2Vec(sentences, vector_size=word_len, window=5, min_count=1, workers=4)
        word2vec_model.save(model_path)
    return word2vec_model

In [ ]:
def get_dataloader(data_path, seq_len, word_len, batch_size, model_dir='./model'):
    try:
        data_df = pd.read_csv(data_path)
    except Exception as e:
        print(f"Error loading data from {data_path}: {e}")
        print("Using sample data instead.")
        sample_reviews = [
            '这家酒店环境很好，服务周到。', '房间太小了，性价比不高。', 
            '早餐种类丰富，味道不错。', '隔音效果差，晚上很吵。', 
            '地理位置优越，出行方便。', '设施陈旧，体验不佳。',
            '服务员态度热情，点赞。', '网络信号不稳定。',
            '强烈推荐这家！', '不会再来了。'
        ]
        sample_labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
        data_df = pd.DataFrame({'review': sample_reviews, 'label': sample_labels})

    if 'review' not in data_df.columns or 'label' not in data_df.columns:
        raise ValueError("DataFrame必须包含'review'和'label'列。")

    data_df = data_df.dropna(subset=['review', 'label'], axis=0, how='any')
    data_df['review'] = data_df['review'].astype(str)
    data_df['label'] = data_df['label'].astype(int)

    if data_df.empty:
        raise ValueError("数据清洗后为空。请检查您的数据文件。")

    word2vec_model = build_word2vec_model(data_df, word_len, model_dir=model_dir)
    
    idx = np.arange(data_df.shape[0])
    np.random.shuffle(idx)
    data_df_shuffled = data_df.iloc[idx]
    
    split_ratio = 0.8
    train_test_split_index = int(data_df_shuffled.shape[0] * split_ratio)

    if train_test_split_index == 0 or train_test_split_index == data_df_shuffled.shape[0]:
        raise ValueError(f"训练/测试分割产生了一个空集合。总样本数：{data_df_shuffled.shape[0]}，分割索引：{train_test_split_index}。请调整数据或分割比例。")
    
    train_data = data_df_shuffled.iloc[:train_test_split_index]
    test_data = data_df_shuffled.iloc[train_test_split_index:]
    
    train_dataset = RNNDataset(train_data, word2vec_model, seq_len, word_len)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0) 
    
    test_dataset = RNNDataset(test_data, word2vec_model, seq_len, word_len)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    print(f"Data loaded: {len(data_df)} total reviews.")
    print(f"Train Dataloader: {len(train_dataloader.dataset)} samples, {len(train_dataloader)} batches.")
    print(f"Test Dataloader: {len(test_dataloader.dataset)} samples, {len(test_dataloader)} batches.")
    
    return train_dataloader, test_dataloader

In [ ]:
import torch.nn as nn # 确保nn被导入
import torch.optim as optim # 确保optim被导入
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns # 确保sns被导入

# 数据加载参数
DATA_FILE_PATH = "data/ChnSentiCorp_htl_all.csv"
SEQ_LEN = 50
WORD_LEN = 100
BATCH_SIZE = 64
MODEL_DIR = './model_w2v'

if not os.path.exists(DATA_FILE_PATH) and DATA_FILE_PATH != "data/sample_reviews.csv":
    print(f"警告: 数据文件 {DATA_FILE_PATH} 未找到。")

In [ ]:
try:
    train_loader, test_loader = get_dataloader(DATA_FILE_PATH, SEQ_LEN, WORD_LEN, BATCH_SIZE, model_dir=MODEL_DIR)
    print(f"成功创建 DataLoader。")
    
    if train_loader and len(train_loader.dataset) > 0:
        print(f"训练集样本数: {len(train_loader.dataset)}")
        sample_batch_text, sample_batch_labels = next(iter(train_loader))
        print(f"一批训练数据的形状: {sample_batch_text.shape}")
        print(f"一批训练标签的形状: {sample_batch_labels.shape}")
    else:
        print("训练数据加载器为空或没有数据。")

    if test_loader and len(test_loader.dataset) > 0:
        print(f"测试集样本数: {len(test_loader.dataset)}")
    else:
        print("测试数据加载器为空或没有数据。")

except Exception as e:
    print(f"创建 DataLoader 时出错: {e}")
    print("请检查 get_dataloader 函数和数据文件路径。")

### 模型定义

#### GRU模型实现

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout_prob, use_embedding=True, vocab_size=None, embedding_dim=None, pad_idx=None):
        super().__init__()
        self.use_embedding = use_embedding
        if self.use_embedding:
            if vocab_size is None or embedding_dim is None:
                raise ValueError("vocab_size and embedding_dim must be provided if use_embedding is True.")
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
            rnn_input_dim = embedding_dim
        else:
            rnn_input_dim = input_dim
            
        self.rnn = nn.GRU(rnn_input_dim, 
                         hidden_dim, 
                         num_layers=n_layers, 
                         bidirectional=True, 
                         batch_first=True,
                         dropout=dropout_prob if n_layers > 1 else 0)
                         
        fc_input_dim = hidden_dim * 2
        self.fc1 = nn.Linear(fc_input_dim, hidden_dim)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)

    def forward(self, text_input):
        if self.use_embedding:
            embedded = self.dropout(self.embedding(text_input))
        else:
            embedded = self.dropout(text_input)
        
        output, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            final_hidden_state = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            final_hidden_state = hidden[-1,:,:]
            
        dense1 = self.fc1(self.dropout(final_hidden_state))
        dense1 = self.batch_norm(dense1)
        dense1 = self.act(dense1)
        output = self.fc2(self.dropout(dense1))
        
        return output

#### LSTM模型实现（带注意力机制）

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_prob, use_embedding=True, vocab_size=None, embedding_dim=None, pad_idx=None):
        super().__init__()
        self.use_embedding = use_embedding
        if self.use_embedding:
            if vocab_size is None or embedding_dim is None:
                raise ValueError("vocab_size and embedding_dim must be provided if use_embedding is True.")
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
            lstm_input_dim = embedding_dim
        else:
            lstm_input_dim = input_dim
        
        self.lstm = nn.LSTM(lstm_input_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           batch_first=True,
                           dropout=dropout_prob if n_layers > 1 else 0)
        
        fc_input_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.attention = nn.Linear(fc_input_dim, 1)
        
        self.fc1 = nn.Linear(fc_input_dim, fc_input_dim // 2)
        self.act = nn.ReLU()
        self.batch_norm = nn.BatchNorm1d(fc_input_dim // 2)
        self.fc2 = nn.Linear(fc_input_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        
    def attention_net(self, lstm_output):
        attn_weights = F.softmax(self.attention(lstm_output), dim=1)
        context = torch.sum(attn_weights * lstm_output, dim=1)
        return context
        
    def forward(self, text_input):
        if self.use_embedding:
            embedded = self.dropout(self.embedding(text_input))
        else:
            embedded = self.dropout(text_input)
        
        lstm_output, (hidden, cell) = self.lstm(embedded)
        attn_output = self.attention_net(lstm_output)
        
        dense1 = self.fc1(self.dropout(attn_output))
        dense1 = self.batch_norm(dense1)
        dense1 = self.act(dense1)
        output = self.fc2(self.dropout(dense1))
        
        return output

### 训练和评估函数

In [ ]:
def train_model(model, iterator, optimizer, criterion, scheduler=None, clip=1.0):
    epoch_loss = 0
    epoch_acc = 0 
    
    model.train()
    
    if not iterator or len(iterator) == 0:
        print("训练迭代器为空，跳过训练。")
        return 0, 0

    for batch_idx, (text_batch, labels_batch) in enumerate(iterator):
        text_batch = text_batch.to(device)
        labels_batch = labels_batch.to(device) 
        
        optimizer.zero_grad()
        predictions = model(text_batch) 
        
        if isinstance(criterion, nn.BCEWithLogitsLoss):
            predictions = predictions.squeeze(-1) if predictions.dim() > 1 and predictions.size(-1) == 1 else predictions
            labels_batch = labels_batch.squeeze(-1).float() if labels_batch.dim() > 1 and labels_batch.size(-1) == 1 else labels_batch.float()
            loss = criterion(predictions, labels_batch)
            rounded_preds = torch.round(torch.sigmoid(predictions))
            correct = (rounded_preds == labels_batch).float()
            acc = correct.sum() / len(correct)
        elif isinstance(criterion, nn.CrossEntropyLoss):
            labels_batch = labels_batch.squeeze(-1).long() if labels_batch.dim() > 1 and labels_batch.size(-1) == 1 else labels_batch.long()
            loss = criterion(predictions, labels_batch)
            top_p, top_class = predictions.topk(1, dim=1)
            correct = (top_class.squeeze(1) == labels_batch).float()
            acc = correct.sum() / len(correct)
        else:
            raise ValueError("不支持的损失函数类型")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    avg_epoch_loss = epoch_loss / len(iterator)
    if scheduler is not None and isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
        scheduler.step(avg_epoch_loss) # ReduceLROnPlateau需要监控指标
    elif scheduler is not None: # 其他类型的调度器
        scheduler.step()
        
    return avg_epoch_loss, epoch_acc / len(iterator)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='best_model.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path
        
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'验证损失减少 ({self.val_loss_min:.6f} --> {val_loss:.6f})。保存模型...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def evaluate_model(model, iterator, criterion):
    epoch_loss = 0
    all_preds_list = []
    all_labels_list = []
    
    model.eval()

    if not iterator or len(iterator) == 0:
        print("评估迭代器为空，跳过评估。")
        return 0, 0, np.array([]), [], []
    
    with torch.no_grad():
        for text_batch, labels_batch in iterator:
            text_batch = text_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            predictions = model(text_batch)
            
            current_labels_np = (labels_batch.squeeze(-1).cpu().numpy() if labels_batch.dim() > 1 and labels_batch.size(-1) == 1 else labels_batch.cpu().numpy())
            all_labels_list.extend(current_labels_np)

            if isinstance(criterion, nn.BCEWithLogitsLoss):
                predictions_squeezed = predictions.squeeze(-1) if predictions.dim() > 1 and predictions.size(-1) == 1 else predictions
                labels_batch_squeezed = labels_batch.squeeze(-1).float() if labels_batch.dim() > 1 and labels_batch.size(-1) == 1 else labels_batch.float()
                loss = criterion(predictions_squeezed, labels_batch_squeezed)
                rounded_preds = torch.round(torch.sigmoid(predictions_squeezed))
                all_preds_list.extend(rounded_preds.cpu().numpy())
            elif isinstance(criterion, nn.CrossEntropyLoss):
                labels_batch_squeezed = labels_batch.squeeze(-1).long() if labels_batch.dim() > 1 and labels_batch.size(-1) == 1 else labels_batch.long()
                loss = criterion(predictions, labels_batch_squeezed)
                top_p, top_class = predictions.topk(1, dim=1)
                all_preds_list.extend(top_class.squeeze(1).cpu().numpy())
            else:
                raise ValueError("不支持的损失函数类型")

            epoch_loss += loss.item()
            
    if not all_labels_list or not all_preds_list:
        print("没有标签或预测结果可用于计算指标。")
        return epoch_loss / len(iterator) if len(iterator) > 0 else 0, 0, np.array([]), [], []

    all_labels_np = np.array(all_labels_list)
    all_preds_np = np.array(all_preds_list)

    final_accuracy = accuracy_score(all_labels_np.astype(int), all_preds_np.astype(int))
    cm = confusion_matrix(all_labels_np.astype(int), all_preds_np.astype(int))
    
    return epoch_loss / len(iterator), final_accuracy, cm, all_labels_np, all_preds_np

In [ ]:
def plot_loss_curves(train_losses, val_losses, title_prefix=""):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='训练损失 (Train Loss)')
    if val_losses: 
        plt.plot(val_losses, label='验证损失 (Validation Loss)')
    plt.title(f'{title_prefix} 训练和验证损失曲线')
    plt.xlabel('轮次 (Epochs)')
    plt.ylabel('损失 (Loss)')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_confusion_matrix(cm, class_names, title_prefix=""):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{title_prefix} 混淆矩阵 (Confusion Matrix)')
    plt.xlabel('预测标签 (Predicted Label)')
    plt.ylabel('真实标签 (True Label)')
    plt.show()

### 模型配置及初始化

In [ ]:
# 全局参数设置
USING_PRECOMPUTED_EMBEDDINGS = True 
EMBEDDING_DIM_MODEL = WORD_LEN # 使用已定义的WORD_LEN
OUTPUT_DIM = 1 # 二分类任务

# GRU (RNNModel) 参数
RNN_INPUT_DIM = EMBEDDING_DIM_MODEL 
HIDDEN_DIM_RNN = 256 
N_LAYERS_RNN = 2     
DROPOUT_RNN = 0.5    

rnn_model = RNNModel(input_dim=RNN_INPUT_DIM,
                     hidden_dim=HIDDEN_DIM_RNN, 
                     output_dim=OUTPUT_DIM, 
                     n_layers=N_LAYERS_RNN, 
                     dropout_prob=DROPOUT_RNN,
                     use_embedding=False, # 因为我们使用预计算的Word2Vec
                    ).to(device)

LEARNING_RATE_RNN = 0.001
WEIGHT_DECAY_RNN = 1e-5
optimizer_rnn = optim.Adam(rnn_model.parameters(), lr=LEARNING_RATE_RNN, weight_decay=WEIGHT_DECAY_RNN)
scheduler_rnn = optim.lr_scheduler.ReduceLROnPlateau(optimizer_rnn, mode='min', factor=0.5, patience=3)
criterion_rnn = nn.BCEWithLogitsLoss().to(device) if OUTPUT_DIM == 1 else nn.CrossEntropyLoss().to(device)
early_stopping_rnn = EarlyStopping(patience=5, verbose=True, path='best_rnn_model.pt')

print("GRU 模型结构:")
print(rnn_model)

In [ ]:
# LSTM (LSTMModel) 参数
LSTM_INPUT_DIM = EMBEDDING_DIM_MODEL
HIDDEN_DIM_LSTM = 256
N_LAYERS_LSTM = 2
BIDIRECTIONAL_LSTM = True
DROPOUT_LSTM = 0.5

lstm_model = LSTMModel(input_dim=LSTM_INPUT_DIM,
                       hidden_dim=HIDDEN_DIM_LSTM, 
                       output_dim=OUTPUT_DIM, 
                       n_layers=N_LAYERS_LSTM, 
                       bidirectional=BIDIRECTIONAL_LSTM, 
                       dropout_prob=DROPOUT_LSTM,
                       use_embedding=False, # 因为我们使用预计算的Word2Vec
                      ).to(device)

LEARNING_RATE_LSTM = 0.001
WEIGHT_DECAY_LSTM = 1e-5
optimizer_lstm = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE_LSTM, weight_decay=WEIGHT_DECAY_LSTM)
scheduler_lstm = optim.lr_scheduler.ReduceLROnPlateau(optimizer_lstm, mode='min', factor=0.5, patience=3)
criterion_lstm = nn.BCEWithLogitsLoss().to(device) if OUTPUT_DIM == 1 else nn.CrossEntropyLoss().to(device)
early_stopping_lstm = EarlyStopping(patience=5, verbose=True, path='best_lstm_model.pt')

print("LSTM 模型结构:")
print(lstm_model)

### 模型训练

In [ ]:
# 训练GRU模型
N_EPOCHS = 30
rnn_train_losses = []
rnn_val_losses = [] # 用于早停和学习率调度

print("开始训练 GRU 模型...")
if not list(train_loader):
    print("错误：训练数据加载器为空。无法开始训练。请检查数据预处理步骤。")
else:
    for epoch in range(N_EPOCHS):
        train_loss, train_acc = train_model(rnn_model, train_loader, optimizer_rnn, criterion_rnn, scheduler=None) # 训练时不直接用scheduler_rnn
        val_loss, val_acc, _, _, _ = evaluate_model(rnn_model, test_loader, criterion_rnn) # 使用test_loader作为验证集
        
        rnn_train_losses.append(train_loss)
        rnn_val_losses.append(val_loss)
        
        print(f'GRU Epoch: {epoch+1:02}')
        print(f'\t训练损失: {train_loss:.3f} | 训练准确率: {train_acc*100:.2f}%')
        print(f'\t验证损失: {val_loss:.3f} | 验证准确率: {val_acc*100:.2f}%')
        
        scheduler_rnn.step(val_loss) # 基于验证损失调整学习率
        early_stopping_rnn(val_loss, rnn_model)
        if early_stopping_rnn.early_stop:
            print("早停触发")
            break
    print("GRU 模型训练完成。")
    # 加载最佳模型
    if os.path.exists(early_stopping_rnn.path):
        print(f"加载最佳GRU模型: {early_stopping_rnn.path}")
        rnn_model.load_state_dict(torch.load(early_stopping_rnn.path))

In [ ]:
# 训练LSTM模型
lstm_train_losses = []
lstm_val_losses = [] # 用于早停和学习率调度

print("开始训练 LSTM 模型...")
if not list(train_loader):
    print("错误：训练数据加载器为空。无法开始训练。")
else:
    for epoch in range(N_EPOCHS):
        train_loss, train_acc = train_model(lstm_model, train_loader, optimizer_lstm, criterion_lstm, scheduler=None)
        val_loss, val_acc, _, _, _ = evaluate_model(lstm_model, test_loader, criterion_lstm)
        
        lstm_train_losses.append(train_loss)
        lstm_val_losses.append(val_loss)
        
        print(f'LSTM Epoch: {epoch+1:02}')
        print(f'\t训练损失: {train_loss:.3f} | 训练准确率: {train_acc*100:.2f}%')
        print(f'\t验证损失: {val_loss:.3f} | 验证准确率: {val_acc*100:.2f}%')
        
        scheduler_lstm.step(val_loss)
        early_stopping_lstm(val_loss, lstm_model)
        if early_stopping_lstm.early_stop:
            print("早停触发")
            break
    print("LSTM 模型训练完成。")
    if os.path.exists(early_stopping_lstm.path):
        print(f"加载最佳LSTM模型: {early_stopping_lstm.path}")
        lstm_model.load_state_dict(torch.load(early_stopping_lstm.path))

### 模型评估与可视化

In [ ]:
# 绘制损失曲线
plot_loss_curves(rnn_train_losses, rnn_val_losses, title_prefix="GRU")
plot_loss_curves(lstm_train_losses, lstm_val_losses, title_prefix="LSTM")

In [ ]:
# 评估GRU模型
test_loss_rnn, test_acc_rnn, cm_rnn = 0.0, 0.0, np.array([])
if not list(test_loader) or len(test_loader.dataset) == 0:
    print("错误：测试数据加载器为空或不包含数据。无法进行评估。")
elif 'rnn_model' not in locals() or not rnn_train_losses:
    print("错误：GRU模型未定义或未训练，无法评估。")
else:
    test_loss_rnn, test_acc_rnn, cm_rnn, _, _ = evaluate_model(rnn_model, test_loader, criterion_rnn)
    print(f'GRU 测试损失: {test_loss_rnn:.3f} | GRU 测试准确率: {test_acc_rnn*100:.2f}%')

    class_names = ['负面 (0)', '正面 (1)']
    if cm_rnn.size > 0:
        plot_confusion_matrix(cm_rnn, class_names, title_prefix="GRU")
    else:
        print("GRU混淆矩阵为空，不绘制。")

In [ ]:
# 评估LSTM模型
test_loss_lstm, test_acc_lstm, cm_lstm = 0.0, 0.0, np.array([])
if not list(test_loader) or len(test_loader.dataset) == 0:
    print("错误：测试数据加载器为空或不包含数据。无法进行评估。")
elif 'lstm_model' not in locals() or not lstm_train_losses:
    print("错误：LSTM模型未定义或未训练，无法评估。")
else:
    test_loss_lstm, test_acc_lstm, cm_lstm, _, _ = evaluate_model(lstm_model, test_loader, criterion_lstm)
    print(f'LSTM 测试损失: {test_loss_lstm:.3f} | LSTM 测试准确率: {test_acc_lstm*100:.2f}%')
    
    class_names = ['负面 (0)', '正面 (1)'] # 确保class_names已定义
    if cm_lstm.size > 0:
        plot_confusion_matrix(cm_lstm, class_names, title_prefix="LSTM")
    else:
        print("LSTM 混淆矩阵为空，不绘制。")

In [ ]:
# 性能对比
print("性能对比:")
if 'test_acc_rnn' in locals() and 'test_acc_lstm' in locals():
    print(f"GRU 模型测试准确率: {test_acc_rnn*100:.2f}%")
    print(f"LSTM 模型测试准确率: {test_acc_lstm*100:.2f}%")

    if test_acc_lstm > test_acc_rnn:
        print("LSTM 模型在此任务和超参数设置下表现更好。")
    elif test_acc_rnn > test_acc_lstm:
        print("GRU 模型在此任务和超参数设置下表现更好。")
    else:
        print("GRU 和 LSTM 模型表现相当。")
else:
    print("未能完成两个模型的评估，无法进行比较。请检查之前的步骤。")

In [ ]:
# 模型性能可视化对比
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
if rnn_train_losses and lstm_train_losses: # 确保列表非空
    # 确保epochs长度与最短的loss列表匹配，以防早停导致长度不一
    min_epochs = min(len(rnn_train_losses), len(lstm_train_losses))
    epochs_range = range(1, min_epochs + 1)
    plt.plot(epochs_range, rnn_train_losses[:min_epochs], 'b-', label='GRU Train Loss')
    plt.plot(epochs_range, lstm_train_losses[:min_epochs], 'r-', label='LSTM Train Loss')
    if rnn_val_losses and lstm_val_losses:
        min_val_epochs = min(len(rnn_val_losses), len(lstm_val_losses))
        val_epochs_range = range(1, min_val_epochs + 1)
        plt.plot(val_epochs_range, rnn_val_losses[:min_val_epochs], 'b--', label='GRU Val Loss')
        plt.plot(val_epochs_range, lstm_val_losses[:min_val_epochs], 'r--', label='LSTM Val Loss')
else:
    print("训练损失数据不完整，无法绘制损失对比图。")
plt.title('训练和验证损失对比')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
model_names = ['GRU', 'LSTM']
accuracies = [test_acc_rnn * 100, test_acc_lstm * 100]
bars = plt.bar(model_names, accuracies, color=['blue', 'red'])
plt.title('测试准确率对比')
plt.ylabel('准确率 (%)')
plt.ylim([0, 100])

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{height:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# 绘制混淆矩阵对比
if 'cm_rnn' in locals() and 'cm_lstm' in locals() and cm_rnn.size > 0 and cm_lstm.size > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    class_names = ['负面 (0)', '正面 (1)'] # 确保class_names已定义
    
    sns.heatmap(cm_rnn, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, 
                yticklabels=class_names, ax=ax1, cbar=False)
    ax1.set_title('GRU 混淆矩阵')
    ax1.set_xlabel('预测标签')
    ax1.set_ylabel('真实标签')
    
    sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Reds', xticklabels=class_names, 
                yticklabels=class_names, ax=ax2, cbar=False)
    ax2.set_title('LSTM 混淆矩阵')
    ax2.set_xlabel('预测标签')
    ax2.set_ylabel('真实标签')
    
    plt.tight_layout()
    plt.show()
else:
    print("混淆矩阵数据不完整，无法绘制对比图。")

In [ ]:
# 模型预测示例分析
def predict_samples(model, data_loader, num_samples=5):
    model.eval()
    all_samples = []
    all_predictions = []
    all_labels = []
    samples_collected = 0
    
    with torch.no_grad():
        for batch in data_loader:
            text, label = batch
            batch_size = text.size(0)
            samples_to_take = min(batch_size, num_samples - samples_collected)
            if samples_to_take <= 0:
                break
            
            text_subset = text[:samples_to_take].to(device)
            label_subset = label[:samples_to_take]
            output = model(text_subset)
            
            if output.shape[1] == 1 if len(output.shape) > 1 else False:
                pred = torch.round(torch.sigmoid(output)).cpu()
            else:
                pred = output.argmax(dim=1, keepdim=True).cpu() if len(output.shape) > 1 else output.unsqueeze(1).cpu()
            
            for i in range(samples_to_take):
                all_samples.append(text_subset[i].cpu())
                all_predictions.append(pred[i])
                all_labels.append(label_subset[i])
            
            samples_collected += samples_to_take
            if samples_collected >= num_samples:
                break
    return all_samples, all_predictions, all_labels

In [ ]:
if list(test_loader) and len(test_loader.dataset) > 0:
    if 'rnn_model' in locals() and 'lstm_model' in locals():
        # 确保测试加载器至少有5个批次，或者调整num_samples
        # 注意：predict_samples现在从批次中取样本，所以len(test_loader)是批次数
        # 我们需要确保有足够的样本，而不是批次
        total_test_samples = len(test_loader.dataset)
        num_display_samples = min(5, total_test_samples)
        
        if num_display_samples > 0:
            print("获取GRU模型预测...")
            rnn_samples, rnn_preds, rnn_labels = predict_samples(rnn_model, test_loader, num_display_samples)
            
            print("获取LSTM模型预测...")
            # 注意：predict_samples会重置data_loader的迭代器，或者从头开始迭代
            # 如果data_loader不是可重置的，这可能导致问题，但PyTorch DataLoader通常是可重置的
            lstm_samples, lstm_preds, lstm_labels = predict_samples(lstm_model, test_loader, num_display_samples)
            
            print("\n预测结果对比:")
            print("=" * 50)
            for i in range(len(rnn_samples)):
                print(f"样本 {i+1}:")
                true_label = rnn_labels[i].squeeze().item() if rnn_labels[i].numel() == 1 else rnn_labels[i][0].item()
                rnn_pred_val = rnn_preds[i].squeeze().item() if rnn_preds[i].numel() == 1 else rnn_preds[i][0].item()
                lstm_pred_val = lstm_preds[i].squeeze().item() if lstm_preds[i].numel() == 1 else lstm_preds[i][0].item()
                
                print(f"真实标签: {true_label}")
                print(f"GRU 预测: {rnn_pred_val}, {'正确' if rnn_pred_val == true_label else '错误'}")
                print(f"LSTM 预测: {lstm_pred_val}, {'正确' if lstm_pred_val == true_label else '错误'}")
                print("-" * 50)
                
            if rnn_preds: # 确保列表非空
                agreement_count = sum(1 for rp, lp in zip(rnn_preds, lstm_preds) if rp.item() == lp.item())
                agreement_rate = agreement_count / len(rnn_preds)
                print(f"GRU 和 LSTM 预测一致率: {agreement_rate:.2%}")
        else:
            print("测试集样本不足，无法显示预测示例。")
    else:
        print("GRU或LSTM模型未定义，无法进行预测分析。")
else:
    print("测试数据加载器为空，无法获取样本进行预测分析。")

## 结论与分析

从实验结果可以看出，LSTM模型在这个中文情感分析任务上的表现通常优于GRU模型（具体结果取决于随机种子和训练过程）。

主要原因可能包括：
1. **长依赖处理**：LSTM通过其门控机制（输入门、遗忘门、输出门）能更有效地捕捉和利用文本中的长距离依赖关系，这对于理解整个评论的情感至关重要。
2. **注意力机制**：在LSTM模型中加入的注意力机制使其能够关注评论中的关键部分（对情感判断更重要的词语），而不是平等对待所有词语，这有助于提升性能。
3. **双向结构**：两个模型都使用了双向结构，允许模型同时考虑过去（前文）和未来（后文）的上下文信息，这对情感分析任务非常有益。
4. **模型复杂度**：LSTM通常比GRU有更多的参数，这可能使其有能力学习更复杂的模式，但也可能更容易过拟合（通过Dropout、权重衰减和早停来缓解）。

可能的改进方向：
1. **预训练词向量/语言模型**：使用更强大的预训练中文词向量（如腾讯词向量、百度词向量）或直接使用预训练的中文语言模型（如BERT、ERNIE）进行微调，通常能带来显著性能提升。
2. **超参数调优**：进行更细致的超参数搜索（如隐藏层大小、层数、Dropout比例、学习率等）。
3. **数据增强**：对训练数据进行增强，例如回译、同义词替换等，以增加数据量和多样性。
4. **更复杂的模型结构**：尝试更先进的模型结构，如Transformer编码器、或者结合CNN和RNN/LSTM的混合模型。
5. **优化器和学习率策略**：尝试不同的优化器（如AdamW）和更高级的学习率调度策略。
6. **特征工程**：除了词向量，还可以考虑加入其他特征，如词性、情感词典特征等。